"""
═══════════════════════════════════════════════════════════════════
CV ANALYZER PRO - VERSION JUPYTER COMPLÈTE
═══════════════════════════════════════════════════════════════════

Outil d'analyse de compatibilité CV/Offre d'emploi avec suggestions
intelligentes pour optimiser vos candidatures.

Auteur : Salwa Zaaraoui
Version : 2.0 Pro (Sans API payante)
Date : Janvier 2025

COMMENT UTILISER CE NOTEBOOK :
1. Exécutez les cellules dans l'ordre (Shift + Entrée)
2. Utilisez l'interface interactive OU la méthode manuelle
3. Suivez les recommandations personnalisées
4. Exportez vos résultats

═══════════════════════════════════════════════════════════════════
"""

In [2]:
# ═══════════════════════════════════════════════════════════════════
# CELLULE 1 : INSTALLATION DES DÉPENDANCES
# Exécutez cette cellule UNE SEULE FOIS au début
# ═══════════════════════════════════════════════════════════════════

!pip install PyPDF2 python-docx ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [10]:
# ═══════════════════════════════════════════════════════════════════
# CELLULE 2 : IMPORTS ET CLASSE PRINCIPALE
# Exécutez cette cellule après l'installation
# ═══════════════════════════════════════════════════════════════════

import re
from collections import Counter
from pathlib import Path
from datetime import datetime
import PyPDF2
import docx

class CVAnalyzerPro:
    """Analyseur CV professionnel avec suggestions intelligentes"""
    
    def __init__(self):
        # Mots vides en français
        self.stop_words = {
            'le', 'la', 'les', 'un', 'une', 'des', 'de', 'du', 'et', 'à', 
            'dans', 'pour', 'par', 'avec', 'sur', 'ce', 'qui', 'il', 'elle',
            'au', 'aux', 'ou', 'mais', 'donc', 'or', 'ni', 'car', 'en', 'je',
            'tu', 'nous', 'vous', 'ils', 'elles', 'son', 'sa', 'ses', 'leur',
            'leurs', 'mon', 'ma', 'mes', 'ton', 'ta', 'tes', 'notre', 'votre',
            'est', 'sont', 'être', 'avoir', 'fait', 'faire', 'peut', 'tout',
            'plus', 'bien', 'sans', 'cette', 'ces', 'comme', 'où', 'que'
        }
        
        # Compétences techniques à détecter
        self.tech_skills = {
            'python', 'java', 'javascript', 'sql', 'aws', 'azure', 'gcp',
            'docker', 'kubernetes', 'react', 'angular', 'vue', 'node',
            'mongodb', 'postgresql', 'mysql', 'pandas', 'numpy', 'scikit',
            'tensorflow', 'pytorch', 'spark', 'hadoop', 'kafka', 'git',
            'jenkins', 'ci/cd', 'agile', 'scrum', 'excel', 'powerbi',
            'tableau', 'sagemaker', 'lambda', 'ec2', 's3', 'api', 'rest',
            'graphql', 'machine learning', 'deep learning', 'nlp', 'data',
            'analytics', 'etl', 'cloud', 'linux', 'windows', 'bash',
            'power bi', 'power query', 'dax', 'vba', 'r', 'sas', 'spss',
            'sap', 'salesforce', 'looker', 'qlik', 'redshift', 'snowflake',
            'airflow', 'dbt', 'jira', 'confluence', 'slack', 'teams',
            'fabric', 'lakehouse', 'dataverse', 'statistiques', 'statistics'
        }
        
        # Catégories de compétences
        self.skill_categories = {
            'langages': ['python', 'java', 'javascript', 'r', 'sql'],
            'data_tools': ['pandas', 'numpy', 'spark', 'hadoop'],
            'viz': ['powerbi', 'power bi', 'tableau', 'qlik', 'excel'],
            'ml': ['machine learning', 'deep learning', 'pytorch', 'tensorflow', 'scikit'],
            'cloud': ['aws', 'azure', 'gcp', 'cloud'],
            'etl': ['etl', 'airflow', 'ssis', 'power query'],
            'methods': ['agile', 'scrum']
        }
    
    def extract_text_from_pdf(self, pdf_path):
        """Extrait le texte d'un PDF"""
        try:
            text = ""
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
            return text
        except Exception as e:
            print(f"❌ Erreur PDF: {e}")
            return ""
    
    def extract_text_from_docx(self, docx_path):
        """Extrait le texte d'un Word"""
        try:
            doc = docx.Document(docx_path)
            text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
            return text
        except Exception as e:
            print(f"❌ Erreur DOCX: {e}")
            return ""
    
    def extract_text_from_file(self, file_path):
        """Détecte le type et extrait le texte"""
        file_path = Path(file_path)
        
        if not file_path.exists():
            print(f"❌ Fichier introuvable: {file_path}")
            return ""
        
        suffix = file_path.suffix.lower()
        
        if suffix == '.pdf':
            return self.extract_text_from_pdf(file_path)
        elif suffix in ['.docx', '.doc']:
            return self.extract_text_from_docx(file_path)
        elif suffix == '.txt':
            return file_path.read_text(encoding='utf-8')
        else:
            print(f"❌ Format non supporté: {suffix}")
            return ""
    
    def clean_and_tokenize(self, text):
        """Nettoie et découpe le texte en mots-clés"""
        text = text.lower()
        text = re.sub(r'[^a-zàâäçéèêëîïôùûüÿ0-9\s+#-]', ' ', text)
        words = text.split()
        keywords = [
            word for word in words 
            if word not in self.stop_words and len(word) > 2
        ]
        return keywords
    
    def extract_technical_skills(self, text):
        """Extrait les compétences techniques"""
        text_lower = text.lower()
        found_skills = [skill for skill in self.tech_skills if skill in text_lower]
        return set(found_skills)
    
    def calculate_match_score(self, cv_keywords, job_keywords):
        """Calcule le score de compatibilité"""
        cv_set = set(cv_keywords)
        job_set = set(job_keywords)
        
        if not job_set:
            return 0
        
        common = cv_set.intersection(job_set)
        score = (len(common) / len(job_set)) * 100
        return round(score, 2)
    
    def interpret_score(self, score, is_junior=True):
        """Interprète le score de manière réaliste"""
        if is_junior:
            if score < 8:
                return "❌ PROFIL INCOMPATIBLE", "Ce poste ne correspond pas à votre profil. Passez votre chemin."
            elif score < 12:
                return "⚠️ PROFIL ÉLOIGNÉ", "Adaptez fortement votre CV ou cherchez d'autres opportunités."
            elif score < 18:
                return "✅ PROFIL JUNIOR ACCEPTABLE", "Score NORMAL pour un junior. Adaptez 2-3 éléments et POSTULEZ !"
            elif score < 25:
                return "✅✅ BON MATCH", "Très bon score ! Postulez avec confiance."
            else:
                return "🎯 EXCELLENT MATCH", "Match exceptionnel ! Postulez en priorité !"
        else:
            # Interprétation pour profil confirmé
            if score < 15:
                return "❌ FAIBLE", "Ne correspond pas"
            elif score < 30:
                return "⚠️ MOYEN", "Quelques adaptations nécessaires"
            elif score < 45:
                return "✅ BON", "Bonne correspondance"
            else:
                return "🎯 EXCELLENT", "Correspondance exceptionnelle"
    
    def generate_skill_suggestions(self, missing_skills, cv_text, job_text):
        """Génère des suggestions intelligentes pour les compétences manquantes"""
        suggestions = []
        
        for skill in list(missing_skills)[:10]:  # Top 10
            suggestion = {
                'skill': skill,
                'priority': 'CRITIQUE' if skill in ['sql', 'python', 'excel', 'power bi'] else 'IMPORTANT',
                'how_to_add': self._generate_skill_addition_guide(skill, cv_text, job_text)
            }
            suggestions.append(suggestion)
        
        return suggestions
    
    def _generate_skill_addition_guide(self, skill, cv_text, job_text):
        """Génère un guide pour ajouter une compétence"""
        guides = {
            'sql': {
                'in_skills': '• SQL (PostgreSQL, SQL Server - PGAdmin4, SSMS)\n• Requêtes complexes, jointures, agrégations',
                'in_experience': 'Analysé les données avec Python (Pandas) et SQL (PostgreSQL) pour extraire...'
            },
            'excel': {
                'in_skills': '• Excel avancé (tableaux croisés dynamiques, Power Query, formules complexes)',
                'in_experience': 'Utilisé Excel et Power Query pour préparer et analyser les données...'
            },
            'power bi': {
                'in_skills': '• Power BI (DAX, Power Query, dashboards interactifs)',
                'in_experience': 'Créé des dashboards Power BI (DAX) pour visualiser...'
            },
            'tableau': {
                'in_skills': '• Tableau (création de visualisations et tableaux de bord)',
                'in_experience': 'Développé des visualisations Tableau pour...'
            },
            'statistics': {
                'in_skills': '• Statistiques (descriptives, inférentielles, tests d\'hypothèses)',
                'in_experience': 'Appliqué des analyses statistiques pour identifier...'
            },
            'statistiques': {
                'in_skills': '• Analyse statistique (descriptive, inférentielle)',
                'in_experience': 'Réalisé des analyses statistiques pour...'
            }
        }
        
        return guides.get(skill.lower(), {
            'in_skills': f'• {skill.title()}',
            'in_experience': f'Utilisé {skill} pour...'
        })
    
    def generate_keyword_suggestions(self, missing_keywords, job_counter):
        """Génère des suggestions pour les mots-clés manquants"""
        # Prendre les mots-clés fréquents dans l'offre mais absents du CV
        important_missing = [
            (word, count) for word, count in job_counter.most_common(20)
            if word in missing_keywords and count >= 2 and len(word) > 4
        ]
        
        suggestions = []
        for word, count in important_missing[:10]:
            suggestions.append({
                'keyword': word,
                'frequency': count,
                'suggestion': self._generate_keyword_usage(word)
            })
        
        return suggestions
    
    def _generate_keyword_usage(self, keyword):
        """Génère une suggestion d'utilisation pour un mot-clé"""
        keyword_usage = {
            'analyse': 'Utilisez "analyse de données", "analyse exploratoire", "analyse statistique"',
            'visualisation': 'Mentionnez "visualisation de données avec Power BI/Tableau"',
            'statistiques': 'Ajoutez "analyse statistique" ou "statistiques descriptives/inférentielles"',
            'automatisation': 'Indiquez "automatisation de processus", "automatisation des workflows"',
            'reporting': 'Utilisez "création de rapports", "reporting automatisé"',
            'projet': 'Mentionnez vos "projets data", "gestion de projet"'
        }
        
        return keyword_usage.get(keyword, f'Intégrez le mot "{keyword}" dans vos expériences ou compétences')
    
    def generate_reformulation_suggestions(self, cv_text, job_keywords):
        """Suggère des reformulations basées sur les mots-clés de l'offre"""
        suggestions = []
        
        # Détection de phrases génériques à reformuler
        generic_patterns = [
            (r'implémentation.*outils', 'Développement et configuration de...'),
            (r'gestion.*données', 'Analyse et traitement de données volumineuses avec...'),
            (r'création.*rapport', 'Conception de rapports analytiques et dashboards avec...'),
            (r'optimisation.*processus', 'Optimisation data-driven des processus via...')
        ]
        
        for pattern, suggestion_template in generic_patterns:
            if re.search(pattern, cv_text.lower()):
                suggestions.append({
                    'pattern': pattern.replace('.*', ' '),
                    'suggestion': suggestion_template
                })
        
        return suggestions[:5]  # Top 5
    
    def generate_cover_letter_template(self, cv_skills, job_skills, common_skills, company_name="[Entreprise]"):
        """Génère un template de lettre de motivation personnalisé"""
        
        template = f"""
╔═══════════════════════════════════════════════════════════════════╗
║           TEMPLATE LETTRE DE MOTIVATION PERSONNALISÉE             ║
╚═══════════════════════════════════════════════════════════════════╝

Objet : Candidature Data Analyst - {company_name}

Madame, Monsieur,

§1 - ACCROCHE
Diplômée Business Data Analyst, je me permets de postuler au poste de 
Data Analyst au sein de {company_name}.

§2 - COMPÉTENCES MATCH (Adapté à l'offre)
Mon profil combine des compétences techniques solides en :
"""
        
        # Compétences en commun (top 5)
        if common_skills:
            top_common = list(common_skills)[:5]
            template += "\n• " + "\n• ".join([skill.title() for skill in top_common])
        
        template += """

avec une expérience concrète sur des projets à fort impact.

§3 - PROJET CONCRET (À adapter)
J'ai notamment développé une solution d'intelligence artificielle pour 
l'EPHEC qui a optimisé la gestion des processus pour 5000+ utilisateurs, 
réduisant les délais de traitement de 70%. Ce projet m'a permis de :
• Analyser et traiter 10 000+ données avec Python et SQL
• Créer des dashboards Power BI pour le suivi des KPIs
• Automatiser des workflows complexes
• Démontrer ma capacité à livrer des projets end-to-end

§4 - MOTIVATION (PERSONNALISER ICI !)
[Pourquoi cette entreprise ? Qu'est-ce qui vous attire ?
Mentionnez un projet/valeur de l'entreprise]

§5 - COMPÉTENCES ADDITIONNELLES
"""
        
        # Mentionner quelques compétences clés du job
        if job_skills:
            relevant_skills = [s for s in job_skills if s in cv_skills][:3]
            if relevant_skills:
                template += "Je maîtrise également " + ", ".join(relevant_skills)
        
        template += """, ce qui me permettra 
de contribuer efficacement à vos projets dès mon arrivée.

§6 - CLÔTURE
Je serais ravie d'échanger avec vous pour vous présenter plus en détail 
mon parcours et ma motivation.

Je reste à votre disposition pour un entretien.

Cordialement,

Salwa Zaaraoui
0484083276
zaaraoui.salwa@live.fr
LinkedIn : linkedin.com/in/salwa-zaaraoui

════════════════════════════════════════════════════════════════════

💡 CONSEILS D'UTILISATION :

1. Personnalisez OBLIGATOIREMENT le §4 (Motivation)
2. Adaptez le §3 selon le poste (plus ML, plus BI, plus Data Engineering...)
3. Ajoutez des chiffres spécifiques si possible
4. Gardez la lettre à 1 page maximum (250-300 mots)
5. Relisez pour fautes !

════════════════════════════════════════════════════════════════════
"""
        
        return template
    
    def analyze(self, cv_path, job_description, is_junior=True, export_results=False):
        """Analyse complète avec suggestions intelligentes"""
        
        print("═" * 80)
        print("🔍 CV ANALYZER PRO - ANALYSE COMPLÈTE")
        print("═" * 80)
        
        # 1. Extraction du CV
        print(f"\n📄 Extraction du CV: {cv_path}")
        cv_text = self.extract_text_from_file(cv_path)
        
        if not cv_text:
            print("\n❌ Impossible de lire le CV.")
            return None
        
        print(f"✅ CV extrait ({len(cv_text)} caractères)")
        
        # 2. Analyse
        print("\n🔍 Analyse en cours...")
        cv_keywords = self.clean_and_tokenize(cv_text)
        job_keywords = self.clean_and_tokenize(job_description)
        
        cv_skills = self.extract_technical_skills(cv_text)
        job_skills = self.extract_technical_skills(job_description)
        
        score = self.calculate_match_score(cv_keywords, job_keywords)
        missing_keywords = set(job_keywords) - set(cv_keywords)
        missing_skills = job_skills - cv_skills
        common_skills = cv_skills.intersection(job_skills)
        
        job_counter = Counter(job_keywords)
        
        # 3. Génération des suggestions
        skill_suggestions = self.generate_skill_suggestions(missing_skills, cv_text, job_description)
        keyword_suggestions = self.generate_keyword_suggestions(missing_keywords, job_counter)
        reformulation_suggestions = self.generate_reformulation_suggestions(cv_text, job_keywords)
        
        # 4. Affichage des résultats
        self.display_results_pro(
            score, cv_keywords, job_keywords, cv_skills, job_skills,
            missing_keywords, missing_skills, common_skills, job_counter,
            skill_suggestions, keyword_suggestions, reformulation_suggestions,
            is_junior
        )
        
        # 5. Template lettre de motivation
        print("\n" + "═" * 80)
        print("📧 TEMPLATE LETTRE DE MOTIVATION")
        print("═" * 80)
        cover_letter = self.generate_cover_letter_template(cv_skills, job_skills, common_skills)
        print(cover_letter)
        
        # 6. Export optionnel
        if export_results:
            self.export_analysis(score, skill_suggestions, keyword_suggestions, cover_letter)
        
        return {
            'score': score,
            'cv_skills': cv_skills,
            'job_skills': job_skills,
            'common_skills': common_skills,
            'missing_skills': missing_skills,
            'skill_suggestions': skill_suggestions,
            'keyword_suggestions': keyword_suggestions,
            'cover_letter': cover_letter
        }
    
    def display_results_pro(self, score, cv_kw, job_kw, cv_skills, job_skills,
                           missing_kw, missing_skills, common_skills, job_counter,
                           skill_suggestions, keyword_suggestions, reformulation_suggestions,
                           is_junior=True):
        """Affichage professionnel des résultats"""
        
        print("\n" + "═" * 80)
        print("📊 RÉSULTATS DE L'ANALYSE")
        print("═" * 80)
        
        # Score et interprétation
        status, message = self.interpret_score(score, is_junior)
        print(f"\n🎯 SCORE DE COMPATIBILITÉ: {score}%")
        print(f"   {status}")
        print(f"   {message}")
        
        if is_junior and 8 <= score < 18:
            print(f"\n   📊 CONTEXTE POUR PROFIL JUNIOR:")
            print(f"   ✅ Ce score est NORMAL et ACCEPTABLE")
            print(f"   Les entreprises utilisent des vocabulaires différents")
            print(f"   ")
            print(f"   📈 Benchmarks typiques junior:")
            print(f"      < 8%   : Profil incompatible")
            print(f"      8-12%  : Adapte avant de postuler")
            print(f"      12-18% : ✅ POSTULE ! (avec adaptations)")
            print(f"      18-25% : Très bon match")
            print(f"      25%+   : Excellent (rare)")
        
        # Statistiques
        print(f"\n📈 STATISTIQUES:")
        print(f"   - Mots-clés dans votre CV: {len(set(cv_kw))}")
        print(f"   - Mots-clés dans l'offre: {len(set(job_kw))}")
        print(f"   - Mots-clés en commun: {len(set(cv_kw) & set(job_kw))}")
        
        # Compétences
        print(f"\n💻 COMPÉTENCES TECHNIQUES:")
        print(f"   - Dans votre CV: {len(cv_skills)}")
        print(f"   - Demandées dans l'offre: {len(job_skills)}")
        print(f"   - En commun: {len(common_skills)}")
        
        if common_skills:
            skills_str = ', '.join(sorted(common_skills))
            print(f"\n   ✅ Compétences trouvées:")
            print(f"      {skills_str}")
        
        # SUGGESTIONS INTELLIGENTES
        print("\n" + "═" * 80)
        print("💡 RECOMMANDATIONS PERSONNALISÉES")
        print("═" * 80)
        
        # 1. Compétences critiques à ajouter
        if skill_suggestions:
            print(f"\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
            print(f"1️⃣ COMPÉTENCES CRITIQUES À AJOUTER")
            print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
            
            for i, sugg in enumerate(skill_suggestions[:5], 1):
                skill = sugg['skill']
                priority = sugg['priority']
                how_to = sugg['how_to_add']
                
                print(f"\n❌ {skill.upper()} [{priority}]")
                print(f"   ")
                print(f"   📝 COMMENT L'AJOUTER :")
                print(f"   ")
                print(f"   Dans 'Compétences' :")
                print(f"   {how_to['in_skills']}")
                print(f"   ")
                print(f"   Dans une expérience :")
                print(f"   \"{how_to['in_experience']}\"")
        
        # 2. Mots-clés à intégrer
        if keyword_suggestions:
            print(f"\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
            print(f"2️⃣ MOTS-CLÉS À INTÉGRER")
            print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
            
            for sugg in keyword_suggestions[:5]:
                keyword = sugg['keyword']
                freq = sugg['frequency']
                suggestion = sugg['suggestion']
                
                print(f"\n🔑 \"{keyword}\" (mentionné {freq}× dans l'offre, absent de votre CV)")
                print(f"   → {suggestion}")
        
        # 3. Reformulations suggérées
        if reformulation_suggestions:
            print(f"\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
            print(f"3️⃣ REFORMULATIONS SUGGÉRÉES")
            print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
            
            for sugg in reformulation_suggestions:
                print(f"\n📍 Si votre CV contient : \"{sugg['pattern']}\"")
                print(f"   💡 Reformulez en : \"{sugg['suggestion']}\"")
        
        # Top mots-clés de l'offre
        print(f"\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"4️⃣ TOP MOTS-CLÉS DE L'OFFRE")
        print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        
        top_keywords = job_counter.most_common(15)
        print(f"\n   Vérifiez que ces mots-clés apparaissent dans votre CV :")
        print(f"   ")
        for word, count in top_keywords:
            in_cv = "✅" if word in cv_kw else "❌"
            print(f"   {in_cv} {word} (×{count})")
        
        # Impact estimé
        print(f"\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        print(f"📊 IMPACT ESTIMÉ DES MODIFICATIONS")
        print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
        
        potential_improvement = len(skill_suggestions[:5]) * 2 + len(keyword_suggestions[:5]) * 1
        estimated_new_score = round(min(score + potential_improvement, 30), 2)
        
        print(f"\n   Score actuel           : {score}%")
        print(f"   Score prévu (si tout appliqué) : {estimated_new_score}%")
        print(f"   Amélioration estimée   : +{round(estimated_new_score - score, 1)}%")
        print(f"   ")
        print(f"   Compétences : {len(common_skills)}/{len(job_skills)} → {min(len(common_skills) + len(skill_suggestions[:5]), len(job_skills))}/{len(job_skills)}")
        
        print("\n" + "═" * 80)
        print("✨ Analyse terminée!")
        print("═" * 80)
    
    def export_analysis(self, score, skill_suggestions, keyword_suggestions, cover_letter):
        """Exporte l'analyse dans un fichier texte"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"cv_analysis_{timestamp}.txt"
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write("═" * 80 + "\n")
            f.write("CV ANALYZER PRO - RAPPORT D'ANALYSE\n")
            f.write("═" * 80 + "\n\n")
            f.write(f"Date : {datetime.now().strftime('%d/%m/%Y %H:%M')}\n")
            f.write(f"Score : {score}%\n\n")
            
            f.write("COMPÉTENCES À AJOUTER:\n")
            f.write("-" * 40 + "\n")
            for sugg in skill_suggestions:
                f.write(f"• {sugg['skill'].upper()} [{sugg['priority']}]\n")
            
            f.write("\n\nMOTS-CLÉS À INTÉGRER:\n")
            f.write("-" * 40 + "\n")
            for sugg in keyword_suggestions:
                f.write(f"• {sugg['keyword']}\n")
            
            f.write("\n\n" + cover_letter)
        
        print(f"\n💾 Rapport exporté : {filename}")

print("✅ Classe CVAnalyzerPro chargée avec succès!")
print("📝 Passez à la cellule suivante pour utiliser l'outil")


✅ Classe CVAnalyzerPro chargée avec succès!
📝 Passez à la cellule suivante pour utiliser l'outil


In [11]:
# ═══════════════════════════════════════════════════════════════════
# CELLULE 3 : UTILISATION SIMPLE (Copier-Coller)
# La méthode la plus rapide pour analyser !
# ═══════════════════════════════════════════════════════════════════

# Créer l'analyseur
analyzer = CVAnalyzerPro()

# METS LE NOM DE TON FICHIER CV ICI
cv_filename = "zaaraoui_salwa_data_analyste.pdf"  # ⬅️ CHANGE ICI

# COLLE TON OFFRE D'EMPLOI ICI (entre les triple guillemets)
job_description = """
Dans le cadre de son développement, notre client – un cabinet d'huissiers de justice reconnu – recherche un Employé administratif orienté IT fr/nl- Employé administratif avec un excellent sens pratique de l'informatique / Analyste IT h/f/x.Ce poste n'exige pas un diplôme ou une formation en IT, mais bien une grande aisance avec les outils digitaux, les logiciels métiers, et la manipulation de données.

Vos responsabilités en tant qu'Employé administratif orienté IT fr/nl - Employé administratif avec un excellent sens pratique de l'informatique / Analyste IT h/f/x:

Gérer l'administration quotidienne avec efficacité (courriers, saisie, classement, mise à jour de données,
Utiliser, adapter et optimiser les outils internes (modification des index dans les systèmes, mise à jour des taux,
Créer et structurer des rapports personnalisés à partir des systèmes internes
Analyser des données (extraire des chiffres, les ventiler selon des critères définis,
Collaborer avec les équipes pour comprendre leurs besoins informatiques et apporter des solutions concrètes via les outils disponibles
S'assurer de la cohérence et de la qualité des données dans le système

Profil recherché pour cette fonction d'Employé administratif orienté IT fr/nl - Employé administratif avec un excellent sens pratique de l'informatique / Analyste IT h/f/x:

Bilingue FR/NL obligatoire
Très bonne maîtrise des outils bureautiques classiques (Word, Excel, Outlook)
Vous êtes un "touche-à-tout" numérique, à l'aise avec les systèmes informatiques, capable de naviguer dans des logiciels complexes sans formation poussée
Autonome, rigoureux(se), structuré(e), curieux(se)
Capacité à comprendre des logiques techniques (modifications de paramètres, traitements de données, tableaux croisés,
Une première expérience dans une fonction administrative est un atout, mais ce poste peut aussi convenir à un(e) jeune diplômé(e) avec un esprit logique et une vraie facilité avec l'outil informatique

Voici l'offre proposée pour cette focntion d'Employé administratif orienté IT nl/fr- Employé administratif avec un excellent sens pratique de l'informatique / Analyste IT h/f/x:

Type de contrat : CDI à pourvoir après une période d'intérim.
Salaire : 2400€ brut pendant la période d'intérim.
Horaires : Temps plein, horaires classiques de bureau.
Lieu : Bureau situé à Nivelles.
"""  # ⬅️ COLLE TON OFFRE ICI

# LANCE L'ANALYSE
print("🚀 Démarrage de l'analyse...\n")
results = analyzer.analyze(
    cv_filename, 
    job_description,
    is_junior=True,      # ⬅️ Mets False si profil confirmé
    export_results=False  # ⬅️ Mets True pour exporter en fichier texte
)


🚀 Démarrage de l'analyse...

════════════════════════════════════════════════════════════════════════════════
🔍 CV ANALYZER PRO - ANALYSE COMPLÈTE
════════════════════════════════════════════════════════════════════════════════

📄 Extraction du CV: zaaraoui_salwa_data_analyste.pdf
✅ CV extrait (3789 caractères)

🔍 Analyse en cours...

════════════════════════════════════════════════════════════════════════════════
📊 RÉSULTATS DE L'ANALYSE
════════════════════════════════════════════════════════════════════════════════

🎯 SCORE DE COMPATIBILITÉ: 13.1%
   ✅ PROFIL JUNIOR ACCEPTABLE
   Score NORMAL pour un junior. Adaptez 2-3 éléments et POSTULEZ !

   📊 CONTEXTE POUR PROFIL JUNIOR:
   ✅ Ce score est NORMAL et ACCEPTABLE
   Les entreprises utilisent des vocabulaires différents
   
   📈 Benchmarks typiques junior:
      < 8%   : Profil incompatible
      8-12%  : Adapte avant de postuler
      12-18% : ✅ POSTULE ! (avec adaptations)
      18-25% : Très bon match
      25%+   : Excellent (r

In [5]:
# ═══════════════════════════════════════════════════════════════════
# CELLULE 4 : INTERFACE INTERACTIVE (Alternative avec widgets)
# Une interface plus user-friendly !
# ═══════════════════════════════════════════════════════════════════

from ipywidgets import FileUpload, Textarea, Button, VBox, HTML, Output, Dropdown
from IPython.display import display, clear_output

# Widgets
file_upload = FileUpload(
    accept='.pdf,.docx,.txt',
    multiple=False,
    description='CV:'
)

job_text = Textarea(
    placeholder='Collez ici le texte complet de l\'offre d\'emploi...',
    description='Offre:',
    layout={'width': '90%', 'height': '300px'}
)

profile_type = Dropdown(
    options=[('Profil Junior (0-3 ans)', True), ('Profil Confirmé (3+ ans)', False)],
    value=True,
    description='Profil:',
)

analyze_button = Button(
    description='🚀 ANALYSER',
    button_style='success',
    layout={'width': '200px', 'height': '50px'}
)

export_button = Button(
    description='💾 Exporter résultats',
    button_style='info',
    layout={'width': '200px', 'height': '40px'}
)

output = Output()

# Variables globales pour stocker les résultats
current_results = None

def on_analyze(b):
    """Fonction appelée quand on clique sur Analyser"""
    global current_results
    
    with output:
        clear_output()
        
        if not file_upload.value:
            print("❌ Veuillez d'abord uploader votre CV")
            return
        
        if not job_text.value.strip():
            print("❌ Veuillez d'abord coller l'offre d'emploi")
            return
        
        # Sauvegarder le fichier
        uploaded = file_upload.value[0] if isinstance(file_upload.value, tuple) else file_upload.value
        filename = uploaded['name']
        content = uploaded['content']
        
        with open(filename, 'wb') as f:
            f.write(content)
        
        # Analyser
        analyzer_widget = CVAnalyzerPro()
        current_results = analyzer_widget.analyze(
            filename,
            job_text.value,
            is_junior=profile_type.value,
            export_results=False
        )

def on_export(b):
    """Fonction pour exporter les résultats"""
    global current_results
    
    with output:
        if current_results:
            analyzer_export = CVAnalyzerPro()
            analyzer_export.export_analysis(
                current_results['score'],
                current_results['skill_suggestions'],
                current_results['keyword_suggestions'],
                current_results['cover_letter']
            )
        else:
            print("\n⚠️ Lancez d'abord une analyse avant d'exporter")

analyze_button.on_click(on_analyze)
export_button.on_click(on_export)

# Affichage de l'interface
print("📱 INTERFACE INTERACTIVE CV ANALYZER PRO")
print("═" * 80)
display(VBox([
    HTML("<h2>🎯 CV Analyzer Pro</h2>"),
    HTML("<p>Analysez la compatibilité de votre CV avec une offre d'emploi</p>"),
    HTML("<hr>"),
    HTML("<h3>1️⃣ Uploadez votre CV</h3>"),
    file_upload,
    HTML("<h3>2️⃣ Sélectionnez votre profil</h3>"),
    profile_type,
    HTML("<h3>3️⃣ Collez l'offre d'emploi</h3>"),
    job_text,
    HTML("<h3>4️⃣ Lancez l'analyse</h3>"),
    analyze_button,
    export_button,
    HTML("<hr>"),
    output
]))


📱 INTERFACE INTERACTIVE CV ANALYZER PRO
════════════════════════════════════════════════════════════════════════════════


In [9]:
# ═══════════════════════════════════════════════════════════════════
# CELLULE 5 : ANALYSE BATCH (Plusieurs offres d'un coup)
# Utile pour comparer plusieurs offres rapidement
# ═══════════════════════════════════════════════════════════════════

def batch_analysis(cv_filename, job_offers_dict):
    """
    Analyse un CV contre plusieurs offres
    
    job_offers_dict = {
        'Entreprise 1': 'texte offre 1',
        'Entreprise 2': 'texte offre 2',
        ...
    }
    """
    analyzer_batch = CVAnalyzerPro()
    results = []
    
    print("═" * 80)
    print("📊 ANALYSE BATCH - COMPARAISON MULTI-OFFRES")
    print("═" * 80)
    
    for company, job_offer in job_offers_dict.items():
        print(f"\n🏢 Analyse : {company}")
        print("-" * 80)
        
        result = analyzer_batch.analyze(
            cv_filename,
            job_offer,
            is_junior=True,
            export_results=False
        )
        
        results.append({
            'company': company,
            'score': result['score'],
            'common_skills': len(result['common_skills']),
            'missing_skills': len(result['missing_skills'])
        })
        
        print("\n" + "═" * 80)
    
    # Résumé comparatif
    print("\n" + "═" * 80)
    print("📈 RÉSUMÉ COMPARATIF")
    print("═" * 80)
    
    # Trier par score
    results_sorted = sorted(results, key=lambda x: x['score'], reverse=True)
    
    print(f"\n{'Entreprise':<30} {'Score':<10} {'Compétences':<15} {'Priorité'}")
    print("-" * 80)
    
    for r in results_sorted:
        priority = "🎯 Priorité 1" if r['score'] >= 15 else "⚠️ Priorité 2" if r['score'] >= 10 else "❌ Éviter"
        print(f"{r['company']:<30} {r['score']:<10.1f}% {r['common_skills']:<15} {priority}")
    
    return results_sorted

# EXEMPLE D'UTILISATION :
"""
job_offers = {
    'Startup Tech XYZ': 'offre 1...',
    'Banque ABC': 'offre 2...',
    'Consulting DEF': 'offre 3...'
}

results = batch_analysis('mon_cv.pdf', job_offers)
"""


# ═══════════════════════════════════════════════════════════════════
# FIN DU NOTEBOOK
# ═══════════════════════════════════════════════════════════════════

print("""
═══════════════════════════════════════════════════════════════════

✅ CV ANALYZER PRO est prêt !

🎯 COMMENT UTILISER :

• CELLULE 3 : Méthode rapide (copier-coller)
• CELLULE 4 : Interface interactive (widgets)
• CELLULE 5 : Analyse batch (plusieurs offres)

💡 CONSEILS :
• Scores 10-15% = Normal pour junior, POSTULE !
• Suis les suggestions personnalisées
• Adapte 2-3 éléments max par offre
• Utilise le template de lettre de motivation

🚀 BON COURAGE POUR TES CANDIDATURES !

═══════════════════════════════════════════════════════════════════
""")


═══════════════════════════════════════════════════════════════════

✅ CV ANALYZER PRO est prêt !

🎯 COMMENT UTILISER :

• CELLULE 3 : Méthode rapide (copier-coller)
• CELLULE 4 : Interface interactive (widgets)
• CELLULE 5 : Analyse batch (plusieurs offres)

💡 CONSEILS :
• Scores 10-15% = Normal pour junior, POSTULE !
• Suis les suggestions personnalisées
• Adapte 2-3 éléments max par offre
• Utilise le template de lettre de motivation

🚀 BON COURAGE POUR TES CANDIDATURES !

═══════════════════════════════════════════════════════════════════

